In [1]:
import os
import sys
os.chdir("/home/group2/naTtahN_T2/Tan3010/211023_NhatTan_Test")
sys.path.append("..")

In [2]:
from utilFuncs.modelUtils import *

Current directory: /home/group2/naTtahN_T2/Tan3010/211023_NhatTan_Test
Using device: cuda


In [3]:
# dataset = load_dataset('alt')
# train_dataset = dataset['train']
# test_dataset = dataset['test']
# train_dataset[0]

In [4]:
filesTrain = ['split_train2023_0to10k.vi', 'split_train2023_0to10k.lo', 
              'split_train2023_10kto30k.vi', 'split_train2023_10kto30k.lo',
              'split_train2023_40kto100k.vi', 'split_train2023_40kto100k.lo']
allPathsTrain = findFiles(filesTrain, '../**')

Found 6 paths


In [5]:
allDataTrain = readAllData(allPathsTrain)
train_dataset = createDataset([allDataTrain[0], allDataTrain[2], allDataTrain[4]],
                              [allDataTrain[1], allDataTrain[3], allDataTrain[5]])

Read 6 files


In [63]:
filesEval = ['split_val2023_0to10k.vi', 'split_val2023_0to10k.lo', 
              'split_val2023_10kto30k.vi', 'split_val2023_10kto30k.lo',
              'split_val2023_40kto100k.vi', 'split_val2023_40kto100k.lo']
allPathsEval = findFiles(filesEval, '../**')

Found 6 paths


In [64]:
allDataEval = readAllData(allPathsEval)
eval_dataset = createDataset([allDataEval[0], allDataEval[2], allDataEval[4]],
                             [allDataEval[1], allDataEval[3], allDataEval[5]])

Read 6 files


In [6]:
TOKENIZER_REPO = 'facebook/m2m100_418M'
MODEL_REPO = TOKENIZER_REPO

tokenizer, model = loadTokenizerAndSeq2SeqLM(TOKENIZER_REPO, MODEL_REPO,
                                             use_pretrained = False)

MAX_SEQ_LEN = model.config.max_length
set_tokenizer_lang = True
add_lang_token = False

In [46]:
sampleInputSentence = 'Phiên dịch tiếng Lào: câu này sẽ được dịch thành tiếng Lào.'
sampleOutputSentence = 'ການແປພາສາລາວ: ປະໂຫຍກນີ້ຈະຖືກແປເປັນພາສາລາວ.'
toyDataset = createDataset([[sampleInputSentence]], [])

tokenizerOutput = format_translation_data(toyDataset['translation'][0],
                                          tokenizer, set_tokenizer_lang, add_lang_token, 
                                          input_lang = 'vi', target_lang = 'lo', random_lang = False,
                                          MAX_SEQ_LEN = MAX_SEQ_LEN,
                                          )
print(tokenizerOutput[0])
print(tokenizer.convert_ids_to_tokens(tokenizerOutput[0]))
print(tokenizer.decode(tokenizerOutput[0]))

model.eval()
modelOutput = model(tokenizerOutput[0].unsqueeze(0),
                    attention_mask = tokenizerOutput[2].unsqueeze(0),
                    labels = tokenizerOutput[1].unsqueeze(0))

modelGenerate = model.generate(tokenizerOutput[0].unsqueeze(0), max_new_tokens = MAX_SEQ_LEN, forced_bos_token_id = tokenizer.get_lang_id('lo'))
print(modelGenerate)

output_text = tokenizer.decode(modelGenerate[0])
print(tokenizer.convert_ids_to_tokens(modelGenerate[0]))
print(output_text)

tensor([128097,  73158,   6152,  13056,  15183,     98,  20232,      9,  25573,
          2752,   3271,   1304,  13056,   4647,  15183,     98,  20232,      5,
             2,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1, 

In [42]:
tokenizerOutput = format_translation_data(toyDataset['translation'][0],
                                          tokenizer, set_tokenizer_lang, add_lang_token, 
                                          input_lang = 'vi', target_lang = 'lo', random_lang = False,
                                          MAX_SEQ_LEN = MAX_SEQ_LEN,
                                          )

tokens = tokenizer.convert_ids_to_tokens(tokenizerOutput[0])
print(tokens) # Make sure that the special translation token is not 'fragmented'

['__vi__', '▁Phi', 'ên', '▁dịch', '▁tiếng', '▁L', 'ào', ':', '▁câu', '▁này', '▁sẽ', '▁được', '▁dịch', '▁thành', '▁tiếng', '▁L', 'ào', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'

In [49]:
# Testing `data_transform`
toyDataset = createDataset([[sampleInputSentence]], [[sampleOutputSentence]])
in_ids, out_ids, attention_mask = format_translation_data(toyDataset['translation'][0],
                                                          tokenizer, set_tokenizer_lang, add_lang_token, 
                                                          input_lang = 'vi', target_lang = 'lo', random_lang = False,
                                                          MAX_SEQ_LEN = MAX_SEQ_LEN,
                                                          )

print(' '.join(tokenizer.convert_ids_to_tokens(in_ids)))
print(' '.join(tokenizer.convert_ids_to_tokens(out_ids)))

# Testing data generator
data_gen = get_data_generator(
    train_dataset,
    tokenizer, set_tokenizer_lang, add_lang_token,
    MAX_SEQ_LEN,
    input_lang = 'vi', target_lang = 'lo', random_lang = False,
)
data_batch = next(data_gen)
print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)
print('Attention mask shape:', data_batch[2].shape)

__vi__ ▁Phi ên ▁dịch ▁tiếng ▁L ào : ▁câu ▁này ▁sẽ ▁được ▁dịch ▁thành ▁tiếng ▁L ào . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

In [50]:
# Constants
n_epochs = 25
batch_size = 32

print_freq = 100
checkpoint_freq = 500

lr = 7.5e-4

n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.2)

In [51]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps
)

/home/group2/miniconda3/envs/pytorch_Tan3010/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [59]:
model_name = 'M2M100_418M'
n_direction = 2
data_description = 'AllCleaned'
model_checkpoint = ('%s_%dD_%s_batch%d_checkpoint.pt' 
                    % (model_name, n_direction, data_description, batch_size))
model_path = ('%s_%dD_%s_batch%d_fineTunedEpoch{}.pt' 
                    % (model_name, n_direction, data_description, batch_size))

In [61]:
input_lang = None
target_lang = None
random_lang = True

In [65]:
losses, evalLosses = train(model,
                           tokenizer, set_tokenizer_lang, add_lang_token,
                           MAX_SEQ_LEN,
                           input_lang, target_lang, random_lang,
                           batch_size,
                           optimizer, scheduler,
                           n_epochs, n_batches,
                           train_dataset, eval_dataset,
                           print_freq, checkpoint_freq,
                           model_checkpoint, model_path)

  0%|          | 0/2391 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacty of 15.73 GiB of which 35.38 MiB is free. Process 139974 has 9.04 GiB memory in use. Process 256674 has 2.37 GiB memory in use. Including non-PyTorch memory, this process has 4.03 GiB memory in use. Of the allocated memory 3.74 GiB is allocated by PyTorch, and 155.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
plotLoss(losses, smoothing_window_size = 50)

In [ ]:
plotLoss(evalLosses, smoothing_window_size = 1)

In [ ]:
eval_model(model, eval_dataset)